# 复杂问题生成Prompt


复杂问题分为两类：
1. 不同实体比较相同属性。
2. 相同实体比较不同属性。

造数据时除了这两类，还需要添加负样例，给出问题和不相关的文本，让模型拒绝回答，否则如果都是正样本，会变成知识注入，把大量的文档知识都背下来。

## 1 不同实体相同属性

### 1.1 配置环境

In [1]:
# # 导包

# import os

# from groq import Groq
# from dotenv import load_dotenv

# load_dotenv()
# # api = os.environ.get("GROQ_API_KEY")
# api_key = ""
# client = Groq(api_key=api_key)

In [6]:
# def generate_completion(prompt):
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": prompt,
#             }
#         ],
#         # model="llama3-8b-8192",
#         model="gemma2-9b-it",
#     )
#     return chat_completion.choices[0].message.content

# print(generate_completion("如何保持专注"))

In [5]:
from zhipuai import ZhipuAI
import os
 
# 环境变量中获取API Key内容
key = ""
# client = ZhipuAI(
#   api_key=os.environ.get('ZhipuAI-APIKey')
# )


client = ZhipuAI(
  api_key=key)


def generate_completion(prompt):
    response = client.chat.completions.create(
    model='glm-4-flash',
    messages=[
      {'role': 'user', 'content': prompt},
    ],
    stream=False,
  )
    
    return response.choices[0].message.content

print(generate_completion("如何保持专注"))

 

保持专注是提高工作效率和生活质量的重要能力。以下是一些帮助你保持专注的方法：

1. **明确目标**：在开始工作之前，明确你的目标和计划，这有助于你集中精力。

2. **环境整理**：保持工作或学习环境的整洁，减少干扰物，有助于提高专注力。

3. **定时休息**：采用番茄工作法等时间管理技巧，如25分钟专注工作，5分钟休息。

4. **减少干扰**：关闭不必要的通知，如手机、电脑弹窗等。

5. **专注训练**：可以通过冥想、深呼吸等方式训练自己的专注力。

6. **避免多任务**：一次性专注于一项任务，避免同时处理多项任务。

7. **建立奖励机制**：给自己设定奖励，完成任务后给予适当的奖励。

8. **保持健康的生活方式**：良好的睡眠、合理的饮食和适量的运动有助于提高专注力。

9. **培养兴趣**：对所从事的工作或学习内容感兴趣，可以提高专注力。

10. **定期回顾**：定期回顾自己的工作或学习进度，调整计划，保持动力。

通过以上方法，你可以逐步提高自己的专注力，从而在学习和工作中取得更好的成绩。


### 1.2 类别生成

In [9]:
sub_category_prompt = """
给定一个实体类别，请生成该类别的子类别列表。列表的长度取决于类别的范围。
例如，对于小类别，最多可以生成10个，但对于大类别，最多可以生成30个子类别。
请注意，子类别必须是不同的。
给定的类别是：{category}
现在，请使用以下格式为此类别生成子类别列表：[子类别1,子类别2,…,子类别n]
请立即生成
"""
category = "休假"
res = generate_completion(sub_category_prompt.format(category=category))
print(res)

[带薪休假, 公休假, 病假, 事假, 产假, 陪产假, 婚假, 丧假, 年假, 调整休假日, 长期休假, 短期休假, 假日加班补偿, 员工培训假, 妈妈假, 父亲假, 产前检查假, 产后恢复假, 节假日调休, 特殊贡献假, 应急休假, 灾害休假, 军事训练假, 公益性休假, 职业病休假, 心理健康假, 慈善活动假, 国际旅行假, 远程工作假, 晋升前准备假, 研究假, 职业发展假]


In [12]:
sub_categories = res.strip().replace("[", "").replace("]", "").split(",")
sub_categories = [category.strip() for category in sub_categories]
print(sub_categories)

['带薪休假', '公休假', '病假', '事假', '产假', '陪产假', '婚假', '丧假', '年假', '调整休假日', '长期休假', '短期休假', '假日加班补偿', '员工培训假', '妈妈假', '父亲假', '产前检查假', '产后恢复假', '节假日调休', '特殊贡献假', '应急休假', '灾害休假', '军事训练假', '公益性休假', '职业病休假', '心理健康假', '慈善活动假', '国际旅行假', '远程工作假', '晋升前准备假', '研究假', '职业发展假']


### 1.3 问题生成

In [15]:
diff_entity_same_attr_prompt = """
你是一个智能助手，可以按照步骤执行指令。请生成以下格式的输出：
- 类别：请随机选择其中之一： 人物、地区、国家、山脉、河流、组织机构、事件、过程、方法、联赛、产品、设施、车辆、音乐、食物、艺术、书籍、体育、植物、动物……
- 实体1：生成一个随机的该类别实体，它在互联网上的受欢迎程度为{popularity_1}(满分5分)。
- 实体2： 生成另一个随机的该类别实体，它在互联网上的受欢迎程度为{popularity_2}(满分5分)。
- 属性列表：生成一个实体1和实体2的常见属性列表以供比较，用这号分隔。属性可以是教字、类型、方法、描述、定义、程序、子项……
- 选择的属性：从上述生成的属性列表中随机选择一个属性。
- 问题1：针对仅实体1的选择属性提问，问题必须提到实体1。
- 问题2：针对仅实体2的选择属性提问，问题必须提到实体2;问题类型应与问题1相同。
- 知识1：生成一个包含实体1的选择属性及相关信息的8~9个句子的中等长度段落。
- 知识2：生成一个包含实体2的选择属性及相关信息的8~9个句子的中等长度段落。
- 两个实体的问题：生成一个{question_type}问题，询问这两个实体的选择属性问题可能是：属性之间数值比较(更大、相等、更小)、两个属性的差异、属性总结、实体的属性关系等。问题必须提到两个实体名，不要使用这些、那些、这个、那个等指代词或代词。

请现在生成：
类别：{category}
"""
category = "休假"
popularity_1 = 4
popularity_2 = 3
question_type = "what"  # 'what' or 'how'
prompt = diff_entity_same_attr_prompt.format(
        category=category,
        popularity_1=popularity_1,
        popularity_2=popularity_2,
        question_type=question_type,
    )

res = generate_completion(prompt)
print(res)

- 类别：休假
- 实体1：五一劳动节
- 实体2：周末
- 属性列表：放假时长、参与人数、庆祝方式、活动主题、历史意义、组织机构、影响力、参与度
- 选择的属性：放假时长
- 问题1：五一劳动节相对于周末来说，其放假时长如何？
- 问题2：周末的放假时长与五一劳动节相比，有何不同？
- 知识1：五一劳动节是中国的一个重要节日，每年的5月1日庆祝，为期3天，通常有更多的人参与，人们会举行各种庆祝活动，如游行、集会等，具有深厚的文化历史意义。
- 知识2：周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天，适合进行轻松的家庭活动或短途旅行。
- 两个实体的问题：五一劳动节和周末在放假时长上，哪个更长？


In [30]:
import re

pattern = r"^-\s*(.*?)[:：]\s*(.*?)$"
matches = re.findall(pattern, res, re.MULTILINE | re.DOTALL)


question_dict = {
    "category": matches[0][1],
    "entity1":matches[1][1],
    "entity2": matches[2][1],
    "attr_list": matches[3][1],
    "selected_attr": matches[4][1],
    "question1": matches[5][1],
    "question2": matches[6][1],
    "knowledge1": matches[7][1],
    "knowledge2": matches[8][1],
    "question_combined": matches[9][1],
    
}
question_dict

{'category': '休假',
 'entity1': '五一劳动节',
 'entity2': '周末',
 'attr_list': '放假时长、参与人数、庆祝方式、活动主题、历史意义、组织机构、影响力、参与度',
 'selected_attr': '放假时长',
 'question1': '五一劳动节相对于周末来说，其放假时长如何？',
 'question2': '周末的放假时长与五一劳动节相比，有何不同？',
 'knowledge1': '五一劳动节是中国的一个重要节日，每年的5月1日庆祝，为期3天，通常有更多的人参与，人们会举行各种庆祝活动，如游行、集会等，具有深厚的文化历史意义。',
 'knowledge2': '周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天，适合进行轻松的家庭活动或短途旅行。',
 'question_combined': '五一劳动节和周末在放假时长上，哪个更长？'}

### 1.4 子问题答案生成

In [32]:
answer_generation_prompt = """
你是一个智能助手，可以仅根据所提供的知识生成问题的答案。如果你不知道答案或者无法从提供的知识中提取答案，就说不知道，不要试图编造答案。请注意，答案必须基于所提供的知识。

知识片段
{context}

=========

问题:
{question}

=========

现在请按照以下格式生成此问题的答案:
+ Thought:从提供的知识中提取问题的相关信息、细节，然后生成回答问题的推理
+ Answer:根据思考，提供问题的完整答案;如果提供的知识不包含答案或无法推理得到答案，请说明你无法根据你的知识回答此问题

=========

开始生成:

+ Thought:
"""

In [45]:
def parse_answer(text, begin_symbol = "-"):
    """
    Parse the answer from the text.
    """
    pattern = rf"^\s*{begin_symbol}\s**(.*?)[:：]\s*(.*?)$"
    matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL)
    result = {}
    for match in matches:
        key,val = match[0], match[1]
        result[key.strip()] = val.strip()
    return result


In [47]:
answer1_prompt = answer_generation_prompt.format(context = question_dict["knowledge1"], question = question_dict["question1"])
print("prompt:\n",answer1_prompt)
answer1 = generate_completion(answer1_prompt)
print(answer1)
print("answer:\n",answer1)


prompt:
 
你是一个智能助手，可以仅根据所提供的知识生成问题的答案。如果你不知道答案或者无法从提供的知识中提取答案，就说不知道，不要试图编造答案。请注意，答案必须基于所提供的知识。

知识片段
五一劳动节是中国的一个重要节日，每年的5月1日庆祝，为期3天，通常有更多的人参与，人们会举行各种庆祝活动，如游行、集会等，具有深厚的文化历史意义。


问题:
五一劳动节相对于周末来说，其放假时长如何？


现在请按照以下格式生成此问题的答案:
+ Thought:从提供的知识中提取问题的相关信息、细节，然后生成回答问题的推理
+ Answer:根据思考，提供问题的完整答案;如果提供的知识不包含答案或无法推理得到答案，请说明你无法根据你的知识回答此问题


开始生成:

+ Thought:

+ Thought: 从提供的知识中可以看出，五一劳动节是中国的一个公共假期，为期3天。而周末通常是每周末的两天，即周六和周日。因此，我们可以通过比较五一劳动节的放假时长和周末的时长来回答这个问题。

+ Answer: 五一劳动节相对于周末来说，其放假时长更长。五一劳动节放假3天，而周末放假2天。
answer:
 + Thought: 从提供的知识中可以看出，五一劳动节是中国的一个公共假期，为期3天。而周末通常是每周末的两天，即周六和周日。因此，我们可以通过比较五一劳动节的放假时长和周末的时长来回答这个问题。

+ Answer: 五一劳动节相对于周末来说，其放假时长更长。五一劳动节放假3天，而周末放假2天。


In [61]:
def parse_answer(text, begin_symbol = "-"):
    """
    Parse the answer from the text.
    """
    pattern = rf"\{begin_symbol}\s*(.*?)[:：]\s*(.*?)$"
    matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL)
    result = {}
    for match in matches:
        if len(match) != 2: continue
        
        key,val = match[0], match[1]
        result[key.strip()] = val.strip()
    return result


qa1_dict = parse_answer(answer1, begin_symbol="+")
print(qa1_dict)

{'Thought': '从提供的知识中可以看出，五一劳动节是中国的一个公共假期，为期3天。而周末通常是每周末的两天，即周六和周日。因此，我们可以通过比较五一劳动节的放假时长和周末的时长来回答这个问题。', 'Answer': '五一劳动节相对于周末来说，其放假时长更长。五一劳动节放假3天，而周末放假2天。'}


In [63]:
answer2_prompt = answer_generation_prompt.format(context = question_dict["knowledge2"], question = question_dict["question2"])
print("prompt:\n",answer2_prompt)
answer2 = generate_completion(answer2_prompt)
print("answer:\n",answer2)
qa2_dict = parse_answer(answer2, begin_symbol="+")
print(qa2_dict)



prompt:
 
你是一个智能助手，可以仅根据所提供的知识生成问题的答案。如果你不知道答案或者无法从提供的知识中提取答案，就说不知道，不要试图编造答案。请注意，答案必须基于所提供的知识。

知识片段
周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天，适合进行轻松的家庭活动或短途旅行。


问题:
周末的放假时长与五一劳动节相比，有何不同？


现在请按照以下格式生成此问题的答案:
+ Thought:从提供的知识中提取问题的相关信息、细节，然后生成回答问题的推理
+ Answer:根据思考，提供问题的完整答案;如果提供的知识不包含答案或无法推理得到答案，请说明你无法根据你的知识回答此问题


开始生成:

+ Thought:

answer:
 + Thought: 根据提供的知识，我们知道周末的放假时长通常为2天。而五一劳动节是一个长假期，根据我所知，五一劳动节通常有7天的假期。因此，我们可以推断出周末的放假时长与五一劳动节相比是不同的。

+ Answer: 周末的放假时长与五一劳动节相比，不同之处在于周末的放假时长较短，通常为2天，而五一劳动节有7天的假期。
{'Thought': '根据提供的知识，我们知道周末的放假时长通常为2天。而五一劳动节是一个长假期，根据我所知，五一劳动节通常有7天的假期。因此，我们可以推断出周末的放假时长与五一劳动节相比是不同的。', 'Answer': '周末的放假时长与五一劳动节相比，不同之处在于周末的放假时长较短，通常为2天，而五一劳动节有7天的假期。'}


### 1.5 复杂问题答案

In [64]:
final_answer_generation_prompt = """
你是一个智能助手，可以根据所提供的知识生成问题的最终答案。最终的答案必须首先从知识中确定事实，然后推断出问题答案的结论。请注意，你仅根据知识生成最终的答案。
知识片段:
=========
{context}
=========

问题:
{question}

现在请按照以下格式生成问题的最终答案
+Summary:总结知识中的要点
+Thought:根据总结的要点，生成回答问题的思考
+Answer:基于总结的要点，以及思考，生成最终的回答问题的答案
开始生成:
+Summary:
"""

In [65]:
context = f"{question_dict['knowledge1']}\n{question_dict['knowledge2']}"
prompt = final_answer_generation_prompt.format(context=context, question=question_dict["question_combined"])
print("prompt:\n",prompt)
final_answer = generate_completion(prompt)
print("answer:\n",final_answer)


prompt:
 
你是一个智能助手，可以根据所提供的知识生成问题的最终答案。最终的答案必须首先从知识中确定事实，然后推断出问题答案的结论。请注意，你仅根据知识生成最终的答案。
知识片段:
五一劳动节是中国的一个重要节日，每年的5月1日庆祝，为期3天，通常有更多的人参与，人们会举行各种庆祝活动，如游行、集会等，具有深厚的文化历史意义。
周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天，适合进行轻松的家庭活动或短途旅行。

问题:
五一劳动节和周末在放假时长上，哪个更长？

现在请按照以下格式生成问题的最终答案
+Summary:总结知识中的要点
+Thought:根据总结的要点，生成回答问题的思考
+Answer:基于总结的要点，以及思考，生成最终的回答问题的答案
开始生成:
+Summary:

answer:
 +Summary: 根据提供的信息，五一劳动节是在每年的5月1日庆祝，为期3天，具有深厚的文化历史意义。周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天。

+Thought: 从知识片段中可以明确得知五一劳动节的放假时长是3天，而周末的放假时长是2天。因此，通过比较这两个时长，我们可以得出五一劳动节放假时长更长的结论。

+Answer: 五一劳动节在放假时长上比周末更长。


In [71]:
final_answer_dict = parse_answer(final_answer, begin_symbol="+")
print(final_answer_dict)

{'Summary': '根据提供的信息，五一劳动节是在每年的5月1日庆祝，为期3天，具有深厚的文化历史意义。周末是指每周的两天休息日，通常是周六和周日，放假时长较短，通常为2天。', 'Thought': '从知识片段中可以明确得知五一劳动节的放假时长是3天，而周末的放假时长是2天。因此，通过比较这两个时长，我们可以得出五一劳动节放假时长更长的结论。', 'Answer': '五一劳动节在放假时长上比周末更长。'}


### 1.6 合并起来

改成一个函数输出所有的结果

In [73]:
import random 
# category = "休假"
category = "shopping"

sub_category_res = generate_completion(sub_category_prompt.format(category=category))
sub_categories = sub_category_res.strip().replace("[", "").replace("]", "").split(",")
sub_categories = [category.strip() for category in sub_categories]
print(sub_categories)

popularity_1 = random.randint(1, 5)
popularity_2 = random.randint(1, 5)
question_type = random.choice(['what', 'how'])  # 'what' or 'how'

prompt = diff_entity_same_attr_prompt.format(
        category=category,
        popularity_1=popularity_1,
        popularity_2=popularity_2,
        question_type=question_type,
    )

res = generate_completion(prompt)
pattern = r"^-\s*(.*?)[:：]\s*(.*?)$"
matches = re.findall(pattern, res, re.MULTILINE | re.DOTALL)
question_dict = {
    "category": matches[0][1],
    "entity1":matches[1][1],
    "entity2": matches[2][1],
    "attr_list": matches[3][1],
    "selected_attr": matches[4][1],
    "question1": matches[5][1],
    "question2": matches[6][1],
    "knowledge1": matches[7][1],
    "knowledge2": matches[8][1],
    "question_combined": matches[9][1],
    
}
answer1_prompt = answer_generation_prompt.format(context = question_dict["knowledge1"], question = question_dict["question1"])
answer1 = generate_completion(answer1_prompt)
qa1_dict = parse_answer(answer1, begin_symbol="+")

answer2_prompt = answer_generation_prompt.format(context = question_dict["knowledge2"], question = question_dict["question2"])
answer2 = generate_completion(answer2_prompt)
qa2_dict = parse_answer(answer2, begin_symbol="+")


context = f"{question_dict['knowledge1']}\n{question_dict['knowledge2']}"
prompt = final_answer_generation_prompt.format(context=context, question=question_dict["question_combined"])
final_answer = generate_completion(prompt)

final_answer_dict = parse_answer(final_answer, begin_symbol="+")
print(final_answer_dict)
data = {**question_dict, **final_answer_dict}

print(data)

['Online Shopping', 'Department Stores', 'Electronics', 'Fashion', 'Home Appliances', 'Furniture', 'Beauty Products', 'Sports Equipment', 'Toys', 'Groceries', 'Books', 'Music', 'Jewelry', 'Shoes', 'Luggage', 'Car Accessories', 'Health & Fitness', 'Baby Products', 'Pet Supplies', 'Office Supplies', 'Art & Craft', 'Outdoor Gear', 'Books & Magazines', 'Video Games', 'Movies & TV', 'Music Instruments', 'Photography', 'Garden Supplies', 'Party Supplies']
{'Summary': '根据提供的信息，亚马逊作为一个全球性的电子商务平台，其购物体验以便利和商品种类多样性著称，用户可以在数千种商品中进行选择。苹果商店则强调个性化服务和体验式购物，顾客可以亲自试用手中的产品，并享受专业的技术支持和完善的售后服务。然而，在商品种类上，没有直接提及苹果商店的商品种类。', 'Thought': '问题是关于商品种类的丰富程度，从亚马逊的描述中可以推断出它提供数千种商品，这表明其商品种类非常丰富。而关于苹果商店，尽管提供个性化的服务和体验，但未提及具体的商品种类。因此，可以推断亚马逊在商品种类上可能更丰富。', 'Answer': '亚马逊在商品种类上更丰富。'}
{'category': 'shopping', 'entity1': '亚马逊（Amazon）', 'entity2': '苹果商店（Apple Store）', 'attr_list': '营业时间、商品种类、购物环境、购物体验、支付方式、物流服务、会员制度、售后服务', 'selected_attr': '购物体验', 'question1': '亚马逊的购物体验如何？', 'question2': '苹果商店的购物体验如何？', 'knowledge1': '亚马逊作为一个

In [74]:
from pprint import pprint
pprint(data)

{'Answer': '亚马逊在商品种类上更丰富。',
 'Summary': '根据提供的信息，亚马逊作为一个全球性的电子商务平台，其购物体验以便利和商品种类多样性著称，用户可以在数千种商品中进行选择。苹果商店则强调个性化服务和体验式购物，顾客可以亲自试用手中的产品，并享受专业的技术支持和完善的售后服务。然而，在商品种类上，没有直接提及苹果商店的商品种类。',
 'Thought': '问题是关于商品种类的丰富程度，从亚马逊的描述中可以推断出它提供数千种商品，这表明其商品种类非常丰富。而关于苹果商店，尽管提供个性化的服务和体验，但未提及具体的商品种类。因此，可以推断亚马逊在商品种类上可能更丰富。',
 'attr_list': '营业时间、商品种类、购物环境、购物体验、支付方式、物流服务、会员制度、售后服务',
 'category': 'shopping',
 'entity1': '亚马逊（Amazon）',
 'entity2': '苹果商店（Apple Store）',
 'knowledge1': '亚马逊作为一个全球性的电子商务平台，提供多种购物体验，包括在线浏览、快速下单和多样的支付方式。它的购物体验以便利和选择多样性著称，用户可以在数千种商品中进行选择，并通过亚马逊Prime服务享受快速配送。',
 'knowledge2': '苹果商店则提供了一种独特的购物体验，强调个性化服务和体验式购物。顾客可以亲自试用手中的产品，享受到专业的技术支持和完善的售后服务。苹果商店通常位于繁华的商业区，环境优雅，购物体验注重细节和顾客满意度。',
 'question1': '亚马逊的购物体验如何？',
 'question2': '苹果商店的购物体验如何？',
 'question_combined': '亚马逊和苹果商店的购物体验相比，哪一个在商品种类上更丰富？',
 'selected_attr': '购物体验'}


In [75]:
def generate_qa_data(category, popularity_1=None, popularity_2=None, question_type=None):
    # Generate sub-categories
    sub_category_res = generate_completion(sub_category_prompt.format(category=category))
    sub_categories = sub_category_res.strip().replace("[", "").replace("]", "").split(",")
    sub_categories = [category.strip() for category in sub_categories]
    # todo：为什么sub_categories没有利用起来，直接choice，而是让模型选择？

    # Generate popularity and question type
    if popularity_1 is None:
        popularity_1 = random.randint(1, 5)
    if popularity_2 is None:
        popularity_2 = random.randint(1, 5)
    if question_type is None:
        question_type = random.choice(['what', 'how'])  # 'what' or 'how'

    # Create the prompt for different entities with the same attribute
    prompt = diff_entity_same_attr_prompt.format(
        category=category,
        popularity_1=popularity_1,
        popularity_2=popularity_2,
        question_type=question_type,
    )

    # Generate response and extract information
    res = generate_completion(prompt)
    pattern = r"^-\s*(.*?)[:：]\s*(.*?)$"
    matches = re.findall(pattern, res, re.MULTILINE | re.DOTALL)
    
    question_dict = {
        "category": matches[0][1],
        "entity1": matches[1][1],
        "entity2": matches[2][1],
        "attr_list": matches[3][1],
        "selected_attr": matches[4][1],
        "question1": matches[5][1],
        "question2": matches[6][1],
        "knowledge1": matches[7][1],
        "knowledge2": matches[8][1],
        "question_combined": matches[9][1],
    }

    # Generate answers for both entities
    answer1_prompt = answer_generation_prompt.format(context=question_dict["knowledge1"], question=question_dict["question1"])
    answer1 = generate_completion(answer1_prompt)
    qa1_dict = parse_answer(answer1, begin_symbol="+")
    # todo：疑问：answer在哪里起作用了？
    
    answer2_prompt = answer_generation_prompt.format(context=question_dict["knowledge2"], question=question_dict["question2"])
    answer2 = generate_completion(answer2_prompt)
    qa2_dict = parse_answer(answer2, begin_symbol="+")

    # Generate final answer
    context = f"{question_dict['knowledge1']}\n{question_dict['knowledge2']}"
    prompt = final_answer_generation_prompt.format(context=context, question=question_dict["question_combined"])
    final_answer = generate_completion(prompt)

    final_answer_dict = parse_answer(final_answer, begin_symbol="+")
    
    # Combine all data into a single dictionary
    data = {**question_dict, **final_answer_dict}
    
    return data

## 2 相同实体不同属性

## 3 负样本生成